In [1]:
import os

In [2]:
%pwd

'c:\\Users\\fkokr\\Desktop\\MLProject\\Text_Summarization_Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\fkokr\\Desktop\\MLProject\\Text_Summarization_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [7]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config["artifacts_root"]])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config['model_trainer']
        params = self.params['TrainingArguments']
        
        create_directories([config['root_dir']])
           
        model_trainer_config = ModelTrainerConfig(
                root_dir = config['root_dir'],
                data_path = config['data_path'],
                model_ckpt = config['model_ckpt'],
                num_train_epochs = params['num_train_epochs'],
                warmup_steps = params['warmup_steps'],
                per_device_train_batch_size = params['per_device_train_batch_size'],
                weight_decay = params['weight_decay'],
                logging_steps = params['logging_steps'],
                evaluation_strategy = params['evaluation_strategy'],
                eval_steps = params['evaluation_strategy'],
                save_steps = params['save_steps'],
                gradient_accumulation_steps = params['gradient_accumulation_steps']
        )
        
        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\fkokr\Desktop\MLProject\Text_Summarization_Project\venv_nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-24 12:39:35,277:INFO:config:PyTorch version 2.3.0 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        device ="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, 
        #     num_train_epochs=self.config.num_train_epochs, 
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, 
        #     weight_decay=self.config.weight_decay,
        #     logging_steps=self.config.logging_steps, 
        #     evaluation_strategy=self.config.evaluation_strategy, 
        #     eval_steps=self.config.evaluation_strategy,
        #     save_steps=self.config.save_steps, 
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        
        trainer = Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer, 
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()
        
        ## Save Model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsunm-model"))

        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        

In [11]:

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-24 12:39:36,033:INFO:common:YAML file: config\config.yaml loaded successfully.]
[2024-05-24 12:39:36,042:INFO:common:YAML file: params.yaml loaded successfully.]
[2024-05-24 12:39:36,045:INFO:common:Directory artifacts created successfully.]
[2024-05-24 12:39:36,046:INFO:common:Directory artifacts/model_trainer created successfully.]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|█▉        | 10/51 [11:35<46:49, 68.53s/it] 

{'loss': 3.1101, 'grad_norm': 31.95026969909668, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [22:07<31:56, 61.83s/it]

{'loss': 3.0466, 'grad_norm': 15.851120948791504, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9904, 'grad_norm': 20.108272552490234, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [52:25<00:59, 59.99s/it]

{'loss': 2.8546, 'grad_norm': 53.34745407104492, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [53:32<00:00, 62.99s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 3212.6522, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.016, 'train_loss': 3.023369265537636, 'epoch': 1.0}
